1. Modules and functions


In [1]:
import numpy as np
from math import log2

2. Main
2.1. Precision@K

In [2]:
def precission_at_k(y_true: np.array, y_pred: np.array, k: int) -> float:
    """
    y_true: true labels
    y_pred: predicted lables
    k: cutoff length
    """

    if sum(y_true) == 0:
        return -1

    argsort = np.argsort(y_pred)[::-1]
    y_true_sorted = y_true[argsort]
    true_positives = y_true_sorted[:k].sum()

    return true_positives / k

In [3]:
# example array
y_true = np.array([1, 0, 0, 1, 0, 0])
y_pred = np.array([6, 2, 3, 5, 4, 1])

In [4]:
argsort = np.argsort(y_pred)[::-1]
argsort

array([0, 3, 4, 2, 1, 5])

In [5]:
y_true_sorted = y_true[argsort]
y_true_sorted

array([1, 1, 0, 0, 0, 0])

In [6]:
true_positives = y_true_sorted[:6].sum()
true_positives

2

In [7]:
# as expected
precission_at_k(y_true, y_pred, k = 6)

0.3333333333333333

2.2. AP@K, MAP@K







In [8]:
def average_precision(y_true: np.array, y_pred: np.array, k: int) -> float:

    if sum(y_true) == 0:
        return -1

    if len(y_pred) > k:
        y_pred = y_pred[:k]

    argsort = np.argsort(y_pred)[::-1]
    y_true_sorted = y_true[argsort]

    num_hits = 0
    score = 0

    for i, p in enumerate(y_true_sorted, 1):
        if p == 1:
            num_hits += 1
            score += num_hits / i
    if num_hits == 0:
        output = 0
    
    else:
        output = score / min(len(y_true), k)

    return output

In [9]:
average_precision(y_true, y_pred, k = 3)

0.3333333333333333

In [10]:
def mean_average_precision_dict(y_true: dict , y_pred: dict, k: int) -> float:
    test = 0
    for i in y_pred.keys():
        a = average_precision(y_true[i], y_pred[i], k)
        test += a
    output = test/len(y_true)

    return output
    

In [11]:
def mean_average_precision(y_true: np.array , y_pred: np.array, k: int) -> float:
    test = []
    for i in range(y_pred.shape[0]):
        a = average_precision(y_true[i], y_pred[i], k)
        test.append(a)
    output = sum(test) / len(y_true)

    return output

In [12]:
y_true = {1: np.array([0,0,0,1,0,1]),
          2: np.array([1,1,0,1,1,0])}

y_pred = {1: np.array([1,2,4,3,7,3]),
          2: np.array([4,5,2,1,7,8])}

MAP = mean_average_precision_dict(y_true, y_pred, 3)
MAP

0.3333333333333333

MRR 

In [13]:
def reciprocal_rank(y_true: np.array, y_pred: np.array) -> float:
    
    argsort = np.argsort(y_pred)[::-1]
    y_true_sorted = y_true[argsort]
    for i, val in enumerate(y_true_sorted, 1):     
        if val == 1:
            return 1 / i
    return 0

In [14]:
# example array for MRR
y_true = np.array([1, 0, 0, 1, 0, 0])
y_pred = np.array([0, 2, 3, 3.5, 4, 1])

In [15]:
reciprocal_rank(y_true, y_pred)

0.5

2.4. NDCG

In [16]:
def compute_gain(y_value: float, gain_scheme: str) -> float:
    
    gain = {'exp2': 2 ** y_value - 1,
            'const': y_value}

    return float(gain[gain_scheme])

In [17]:
def dcg(y_true: np.array, y_pred: np.array, gain_scheme: str) -> float:
    
    dcg = 0
    argsort = np.argsort(y_pred)[::-1]
    y_true_sorted = y_true[argsort]

    for idx, val in enumerate(y_true_sorted, 1):
        gain = compute_gain(val, gain_scheme)
        dcg += gain / log2(idx + 1)
        
    return dcg

In [24]:
def ndcg(y_true: np.array, ys_pred: np.array, a: int, gain_scheme: str = 'const')  -> float:
    
    
    if len(ys_pred)>a:
        ys_pred_a = ys_pred[:a]
        y_true_a = y_true[:a]
    else:
        ys_pred_a = ys_pred
        y_true_a = y_true
    

    test_dcg = dcg(y_true_a, ys_pred_a, gain_scheme)


    y_true_sort = np.sort(y_true)[::-1]
    y_true_sort_a = y_true_sort[:a]
    dcg_max = dcg(y_true_sort_a, y_true_sort_a, gain_scheme)

    return test_dcg / dcg_max

In [25]:
y_pred = np.array([1, 2, 8, 6, 9, 4]) # some score to sort
y_true = np.array([8, 6, 5,3, 0, 3])

In [29]:
print(ndcg(y_true, y_pred, 3, 'exp2'))

0.6389991470886698
